###  Using SIRIUS to predict formula for standard library

In [156]:
import pandas as pd
import os
import re
def adduct_clean(string):
    if string == '[M + H]+':
        return '[M+H]+'
    elif string == '[M]+':
        return '[M]+'
    elif string == '[M - H2 + H]+':
        return '[M-H]+'
    else:
        return 'others'
    
def formula2dict(formula):
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    return res_dct
def formula2dictH(formula):
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    res_dct['H'] = res_dct['H']+1
    return res_dct
def calculate_mass(formula: str):
    #from yuanyue
    
    mol_mass = 0.
    try:
        all_atom_nums = re.findall('([A-Z][a-z]*)([0-9]*)', formula)
        for atom_num in all_atom_nums:
            n = atom_num[1]
            if n == '':
                mol_mass += atom_mass[atom_num[0]]
            else:
                mol_mass += int(n) * atom_mass[atom_num[0]]
    except KeyError as e:
        print("Atom {} is not known".format(e.args[0]))
    except TypeError:
        print('formula is nan')
    return round(mol_mass,6)

In [157]:
atom_mass = {'H': 1.00782503223,
             'C': 12,
             'N': 14.00307400443,
             'O': 15.99491461957,
             'F': 18.99840316273,
             'P': 30.97376199842,
             'S': 31.9720711744,
             'Cl': 34.968852682,
             'Ar': 39.9623831237,
             'K': 38.9637064864,
             'Ca': 39.962590863,
             'Si': 27.976926535,
             'Na':22.98976928
             }

In [158]:
formula = pd.read_csv('./C18Pos.csv')
formula['short'] = formula.inchikey.str[:14]
formula

,index,name,inchikey,precursormz,formula,adductformula,precursor,short
0,0,ARTEMISININ,BLUAFEHZUWYNDE-NNWCWBAJSA-N,265.14398,C15H22O5,C15H21O4,[M+H-H2O]+,BLUAFEHZUWYNDE
1,1,ARTEMISININ,BLUAFEHZUWYNDE-NNWCWBAJSA-N,247.13342,C15H22O5,C15H19O3,[M+H-H4O2]+,BLUAFEHZUWYNDE
2,2,ARTEMISININ,BLUAFEHZUWYNDE-NNWCWBAJSA-N,283.15455,C15H22O5,C15H23O5,[M+H]+,BLUAFEHZUWYNDE
3,3,RHOIFOLIN,RPMNUQRUHXIGHK-PYXJVEIZSA-N,579.17138,C27H30O14,C27H31O14,[M+H]+,RPMNUQRUHXIGHK
4,4,BERBERINE CHLORIDE,YBHILYKTIRIUTE-UHFFFAOYSA-N,336.12358,C20H18NO4+,C20H18NO4,[M]+,YBHILYKTIRIUTE
...,...,...,...,...,...,...,...,...
14407,14407,CINCHONIDINE,KMPWYEUPVWOPIM-KODHJQJWSA-N,295.18104,C19H22N2O,C19H23N2O,[M+H]+,KMPWYEUPVWOPIM
14408,14408,AURAPTENE,RSDDHGSKLOSQFK-RVDMUPIBSA-N,299.16472,C19H22O3,C19H23O3,[M+H]+,RSDDHGSKLOSQFK
14409,14409,SKYRIN,MQSXZQXHIJMNAF-UHFFFAOYSA-N,521.08726,C30H18O10,C30H17O9,[M+H-H2O]+,MQSXZQXHIJMNAF
14410,14410,SKYRIN,MQSXZQXHIJMNAF-UHFFFAOYSA-N,539.09782,C30H18O10,C30H19O10,[M+H]+,MQSXZQXHIJMNAF


### Read SIRIUS result

In [159]:
path = '../C18Pos_orb/'

In [160]:
rank_list = []
name = []
d_list = []
for directory in os.listdir(path):
    
    #directory = '19_annotate_20'
    
    if '_wo_adduct_' in directory:
        
        index = int(re.findall(r'\d+',directory)[0])
        name.append(index)
        key = formula[formula['index'] == int(index)] # get the key row from forumla
        try:
            predict = pd.read_csv(path+directory+'/formula_candidates.tsv', sep='\t')
            


            predict['dict'] = predict['molecularFormula'].apply(formula2dict)

            keydict = formula2dict(key['formula'].item())
            rank = predict.index[predict['dict'] == keydict].to_list()
            if rank:
                rank_list.append(rank[0])
            else:
                rank_list.append('no correct formula')   # in sirius result 
        except FileNotFoundError:
            rank_list.append('no calculations') # didn't calculate in sirius gui
        except TypeError:
            rank_list.append('no key formula')
            
            
        try:
            predict = pd.read_csv(path+directory+'/structure_candidates.tsv', sep='\t')
            predict = predict.sort_values(by=['CSI:FingerIDScore'], ascending=False).reset_index(drop=True)
            rank = predict.index[predict['InChIkey2D'] == key['short'].item()].to_list()
            if rank:
                d_list.append(rank[0])
            else:
                d_list.append('no correct structure')   # in sirius result 
        except FileNotFoundError:
            d_list.append('no calculations') # didn't calculate in sirius gui
        except TypeError:
            d_list.append('no key structure')
       # break


In [161]:
df = pd.DataFrame({'rank':rank_list,'2d':d_list, 'index':name})
df

,rank,2d,index
0,0,0,0
1,no calculations,no calculations,10000
2,no calculations,no calculations,10001
3,no calculations,no calculations,10002
4,no calculations,no calculations,10003
...,...,...,...
14407,no calculations,no calculations,9998
14408,no calculations,no calculations,9999
14409,0,0,999
14410,0,0,99


In [162]:
formula['mass'] = formula['adductformula'].apply(calculate_mass)
formula = formula.merge(df,on='index',how='left')
formula['ppm'] = abs((formula.mass-formula.precursormz)/formula.precursormz)
formula.to_csv('rank.csv')

Atom Li is not known
Atom I is not known
Atom Li is not known
Atom Li is not known
Atom Li is not known
Atom I is not known
Atom Li is not known
Atom Li is not known
Atom I is not known
Atom I is not known
Atom Br is not known
Atom Br is not known
Atom I is not known
Atom Br is not known
Atom I is not known
Atom Br is not known
Atom I is not known
Atom Br is not known
Atom As is not known
Atom Br is not known
Atom Br is not known
Atom I is not known
Atom Br is not known
Atom I is not known
Atom Li is not known
Atom Li is not known
Atom I is not known
Atom I is not known
Atom I is not known
Atom I is not known
Atom I is not known
Atom I is not known
Atom I is not known
Atom I is not known


In [163]:
formula['2d'].value_counts()

no calculations         8361
no correct structure    4233
0                        827
1                        197
2                         86
                        ... 
281                        1
304                        1
403                        1
244                        1
155                        1
Name: 2d, Length: 196, dtype: int64

In [164]:
formula['rank'].value_counts()

no calculations       8361
no correct formula    3380
0                     1763
1                      582
2                       99
4                       43
3                       43
8                       20
6                       18
5                       18
12                      15
7                       14
11                      13
9                       11
10                      11
13                      10
14                       7
16                       2
15                       1
20                       1
Name: rank, dtype: int64